<a href="https://colab.research.google.com/github/elyager/LLMs-from-scratch/blob/main/LLM_from_scratch_personal_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Requiriments

In [76]:
!pip install tiktoken
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


### Vocabulary

- **vocab_size:** es el tamaño del vocabulario en tokens únicos disponibles+extensiones. En nuestro caso dado por el tokenizador creado con BPE.
- **output_dim:** el número de dimensiones de cada token. Las dimensiones describen a una palabra o concepto. Más dimensiones capturan más detalles.
- **max_length:** es la máxima logitud de tokens por secuencia.
- **batch_size:** es cuantas secuencias (muestras de texto) tiene cada batch. Larger batch sizes can speed up training but might require more memory.
- **stride:** el tamaño de la zancada en tokens, cuantos tokens salta para la siguiente secuencia, esto determina que tanto se empalma una secuencia con otra. Ensure the model sees the context of each token multiple times during training, helping it learn better relationships between words.
- **shuffle:** determina si le da un orden aleatorio a las secuencias para que sea random en cada epoc. Prevents the model from learning patterns based on the order of data presentation. It helps generalize learning and avoid overfitting to a specific data order.
-**drop_last:** indica si se debe descartar el último batch cuando no cumple con el número de muestras establecido en max_lenght. Si se tiene un set de datos pequeño es mejor no hacer drop.
-**num_workers:** es el número de procesos, a mayor número más rápidez.
-**attention score:** determina qué tan "similar" es una pababra con la otra a través de dot product que es un tipo de similarity function. Mayor atention score mayor similitud entre los números.
--**attention weight:** es la versión normalizada a través de softmax de los attention scores.
-**context vector:** es un embedding vector pero que tiene todo el contexto del resto de input vectors. Se obtiene sumando todos los attention weights del inpute secuence.

# Chapter 1 & 2

### Load the text for training (our corpus)

In [77]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### BytePair Encoding (BPE)


In [78]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [79]:
tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
enc_text.append(tokenizer.eot_token)

# First 10 tokens from raw_text
first_10_token_ids = enc_text[:10]
decoded_tokens = [tokenizer.decode([token_id]) for token_id in first_10_token_ids]
delimited_tokens = ' |-|'.join(decoded_tokens)
print(delimited_tokens)
print(enc_text[:10])
print(f'\n Total of tokens: {len(enc_text)}')

I |-| H |-|AD |-| always |-| thought |-| Jack |-| G |-|is |-|burn |-| rather
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]

 Total of tokens: 5146


### Dataset loader (creating tokenIDs for inputs and targets)

In [80]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        token_ids.append(tokenizer.eot_token)

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# dataset = GPTDatasetV1(raw_text, tokenizer, max_length=4, stride=1)
# print(dataset.input_ids)
# print(dataset.target_ids)

In [81]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

### Use DataLoader

In [82]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# for batch in dataloader:
#     input, target = batch
#     print(input, target)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch) # input and target
second_batch = next(data_iter)
print(second_batch) # input and target

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [83]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
# First batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

# Second batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Inputs:
 I HAD always

Targets:
  HAD always thought
Inputs:
 tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]])

Ta

### Create our token embedding layer

In [84]:
vocab_size = 50257
output_dim = 3 # 256 is a more common starting point

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer.weight.shape)
print(token_embedding_layer.weight)

torch.Size([50257, 3])
Parameter containing:
tensor([[ 0.6632, -0.1153, -1.1000],
        [-0.4561, -0.9518,  0.6254],
        [ 0.7506,  0.4049, -0.1069],
        ...,
        [-0.1897, -1.2373, -0.4659],
        [ 1.0391,  1.3802, -0.0861],
        [-0.7439,  1.2514, -0.4061]], requires_grad=True)


#### Load our dataset to get the inputs

In [85]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n",  inputs) # we take the first batch and  ignore the targets for now
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


### Create token embeddings

In [86]:
token_embeddings = token_embedding_layer(inputs)
# each token now has the assigned number of dimentions instead of being a single token ID
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
print(token_embeddings)

torch.Size([8, 4, 3])
tensor([[[-0.3464, -0.3487, -0.3348],
         [ 0.7228,  0.6994,  0.7684],
         [-1.4396, -1.0544,  0.8449],
         [-1.3378,  1.1841,  1.5821]],

        [[ 0.7331,  1.1978,  0.5132],
         [ 0.6777, -0.4469, -0.0546],
         [-0.9882,  0.3909, -1.9161],
         [-0.7682,  1.5992, -0.7182]],

        [[-1.1337,  1.0565,  0.4598],
         [ 0.4786, -1.3459, -1.0433],
         [ 1.1629, -1.5761, -0.2337],
         [ 0.1008, -1.8568, -0.2825]],

        [[-0.8422, -0.5557, -0.6075],
         [ 2.9747,  0.3709,  1.9986],
         [-0.2434, -0.0478, -0.0520],
         [ 1.1629, -1.5761, -0.2337]],

        [[-1.1073, -0.4384,  0.0214],
         [-0.1865,  0.4319, -1.5945],
         [ 0.4360,  0.5448,  0.2600],
         [ 2.9747,  0.3709,  1.9986]],

        [[ 0.3092, -0.2930,  0.9936],
         [-1.8778,  0.3236,  0.2425],
         [-0.8762,  1.8448, -1.2222],
         [ 0.0291,  0.5612, -0.8538]],

        [[ 0.3601, -0.1169, -0.1912],
         [ 0.415

### Create absolute positional embeddings

In [87]:
pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)

# # [0, 1, 2, 3] "column" position is the position of each word on each sequence of 4 context_length
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# # uncomment & execute the following line to see how the embeddings look like
print(pos_embeddings)

torch.Size([4, 3])
tensor([[-0.6064,  0.4270, -2.4227],
        [ 0.1993,  0.8162,  1.4761],
        [-0.6664,  0.3420,  0.1134],
        [ 0.5331,  1.2561, -0.1645]], grad_fn=<EmbeddingBackward0>)


### Create input embeddings

In [88]:
input_embeddings = token_embeddings + pos_embeddings
print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')

# uncomment & execute the following line to see how the embeddings look like
print(input_embeddings)

torch.Size([8, 4, 3]) + torch.Size([4, 3]) = torch.Size([8, 4, 3])
tensor([[[-0.9528,  0.0783, -2.7575],
         [ 0.9221,  1.5156,  2.2446],
         [-2.1060, -0.7123,  0.9583],
         [-0.8048,  2.4402,  1.4176]],

        [[ 0.1267,  1.6248, -1.9095],
         [ 0.8770,  0.3693,  1.4215],
         [-1.6547,  0.7330, -1.8027],
         [-0.2352,  2.8552, -0.8827]],

        [[-1.7401,  1.4834, -1.9630],
         [ 0.6780, -0.5297,  0.4328],
         [ 0.4965, -1.2341, -0.1202],
         [ 0.6339, -0.6007, -0.4469]],

        [[-1.4486, -0.1288, -3.0302],
         [ 3.1740,  1.1870,  3.4747],
         [-0.9098,  0.2942,  0.0615],
         [ 1.6960, -0.3200, -0.3982]],

        [[-1.7137, -0.0115, -2.4013],
         [ 0.0129,  1.2481, -0.1184],
         [-0.2304,  0.8868,  0.3734],
         [ 3.5077,  1.6270,  1.8341]],

        [[-0.2972,  0.1340, -1.4292],
         [-1.6784,  1.1397,  1.7186],
         [-1.5426,  2.1868, -1.1088],
         [ 0.5621,  1.8173, -1.0182]],

        [

### What about more dimmension?

In [89]:
def more_more_more_dimensions(dim):
  vocab_size = 50257
  output_dim = dim

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  print(token_embeddings.shape)
  print(token_embeddings)

  pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(max_length))
  print(pos_embeddings.shape)
  print(pos_embeddings)


  input_embeddings = token_embeddings + pos_embeddings
  print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  print(input_embeddings)

more_more_more_dimensions(256)  #256 is a more common starting point

torch.Size([8, 4, 256])
tensor([[[ 0.7758,  1.3495,  0.5544,  ...,  0.2855, -0.8612,  0.8450],
         [ 0.4139, -0.5915, -0.7529,  ...,  1.2301, -0.9799, -0.4389],
         [-0.3691, -0.7674,  0.8731,  ..., -0.6673, -1.9097,  0.9186],
         [-0.5459,  1.5026, -0.2190,  ..., -0.8375, -0.1477, -1.4801]],

        [[-0.3546,  0.5157,  0.9620,  ..., -0.3160, -0.5081, -0.3975],
         [-0.7932,  1.2741,  0.3305,  ...,  1.0824,  0.9768, -1.0146],
         [-0.3876,  1.5214, -0.5019,  ...,  0.3887,  0.8529, -0.9703],
         [-0.6835, -1.8386, -0.1528,  ...,  1.2304,  1.8741,  0.1236]],

        [[-0.7186, -0.6272,  0.3793,  ...,  0.7871,  0.8277,  0.5450],
         [ 0.8483,  1.3609, -0.4049,  ...,  0.8944, -1.2278, -0.9842],
         [-0.6660,  0.8203,  1.7201,  ..., -0.9344, -0.1927, -1.1092],
         [ 0.6889,  0.4526,  0.6012,  ...,  0.2292,  0.7494, -0.0403]],

        ...,

        [[ 1.9610, -1.0976, -0.0026,  ...,  2.3907, -1.1946, -0.1245],
         [-0.5075,  0.6499,  0.36

# Chapter 3

## Preparing data to work with

### Get input embeddings

In [90]:
def get_input_embeddings(my_raw_text):
  dataloader = create_dataloader_v1(my_raw_text, batch_size=1, max_length=6, stride=6, shuffle=False)

  data_iter = iter(dataloader)
  # First and only batch
  inputs, targets = next(data_iter)
  # print(inputs)
  # print(targets)  # ignore the targets

  vocab_size = 50257
  output_dim = 3

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  context_length = 6
  pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(context_length))

  input_embeddings = token_embeddings + pos_embeddings
  # print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  # print(input_embeddings)
  return input_embeddings

In [91]:
my_raw_text = "Your journey starts with one step"
small_input_embeddings = get_input_embeddings(my_raw_text)[0]
print(small_input_embeddings)

tensor([[-1.4380, -0.7216, -0.0361],
        [ 2.1245, -0.3850, -0.0936],
        [ 0.4108,  1.6764, -0.3740],
        [-1.7692, -2.4817,  0.8789],
        [ 0.8768,  0.8055,  2.0541],
        [ 1.4978,  0.1165, -0.0393]], grad_fn=<SelectBackward0>)


In [92]:
# tensor([
#     [ 0.4113,  1.3397, -1.2234], Your    (x^1)
#     [-1.8881, -0.0679, -1.1267], journey (x^2)
#     [-0.2323, -2.2089, -1.6685], starts  (x^3)
#     [ 0.5615,  1.2698,  2.5768], with    (x^4)
#     [-0.9290, -0.0227,  0.6467], one     (x^5)
#     [ 0.5691, -2.0627, -3.2411]  step    (x^6)
# ])

### Forcing values to fit between 0 and 1

In [93]:
import decimal
min_val = small_input_embeddings.min()
max_val = small_input_embeddings.max()
scaled_embeddings = (small_input_embeddings - min_val) / (max_val - min_val)
rounded_embeddings = torch.round(scaled_embeddings * 100) / 100
print(rounded_embeddings)

tensor([[0.2300, 0.3800, 0.5300],
        [1.0000, 0.4600, 0.5200],
        [0.6300, 0.9000, 0.4600],
        [0.1500, 0.0000, 0.7300],
        [0.7300, 0.7100, 0.9800],
        [0.8600, 0.5600, 0.5300]], grad_fn=<DivBackward0>)


## Simple self-attention

### Step 1 - Compute unormalized attention scores

In [94]:
query = rounded_embeddings[1]  # 2nd input token is the query)
# just allocate a tensor in memory with 6 spaces
attn_scores_2 = torch.empty(rounded_embeddings.shape[0])

#fill the tensor with the dot products which multiply and sum
for i, x_i in enumerate(rounded_embeddings):
    print(f'dot product of {x_i} against journey {query}')
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

dot product of tensor([0.2300, 0.3800, 0.5300], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
dot product of tensor([1.0000, 0.4600, 0.5200], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
dot product of tensor([0.6300, 0.9000, 0.4600], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
dot product of tensor([0.1500, 0.0000, 0.7300], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
dot product of tensor([0.7300, 0.7100, 0.9800], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
dot product of tensor([0.8600, 0.5600, 0.5300], grad_fn=<UnbindBackward0>) against journey tensor([1.0000, 0.4600, 0.5200], grad_fn=<SelectBackward0>)
tensor([0.6804, 1.4820, 1.2832, 0.5296, 1.5662, 1.3932], grad_fn=<CopySlices>)


### Step 2 - Normalize the attenton scores to sum up to 1

In [95]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.0963, 0.2147, 0.1760, 0.0828, 0.2336, 0.1965],
       grad_fn=<DivBackward0>)
Sum: tensor(1.0000, grad_fn=<SumBackward0>)


In [96]:
# Fooling around with dimensions on vectors
my_tensor = torch.ones(2,3,4) # I always start with the last dimension which is [-1]
print(my_tensor)
print(my_tensor.shape)
print(my_tensor.shape[-1]) # last dimension
print(my_tensor.shape[0]) # first dimenson
print(my_tensor.shape[1])
print(my_tensor.shape[2]) # same as [-1]
# print(my_tensor.shape[3]) # IndexError

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
torch.Size([2, 3, 4])
4
2
3
4


In [97]:
# using pytorch softmax fucntion
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum()) #100%


Attention weights: tensor([0.0963, 0.2147, 0.1760, 0.0828, 0.2336, 0.1965],
       grad_fn=<SoftmaxBackward0>)
Sum: tensor(1., grad_fn=<SumBackward0>)


### Step 3 - Compute the context vector $z^{(2)}$

In [98]:
query = rounded_embeddings[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
attn_weights_sum = 0
for i,x_i in enumerate(rounded_embeddings):
    print(f'{attn_weights_2[i]} * {x_i}')
    context_vec_2 += attn_weights_2[i]*x_i
    attn_weights_sum += attn_weights_2[i]

print(attn_weights_sum)
print(context_vec_2)

0.09633011370897293 * tensor([0.2300, 0.3800, 0.5300], grad_fn=<UnbindBackward0>)
0.2147299349308014 * tensor([1.0000, 0.4600, 0.5200], grad_fn=<UnbindBackward0>)
0.17601707577705383 * tensor([0.6300, 0.9000, 0.4600], grad_fn=<UnbindBackward0>)
0.08284579962491989 * tensor([0.1500, 0.0000, 0.7300], grad_fn=<UnbindBackward0>)
0.2335931658744812 * tensor([0.7300, 0.7100, 0.9800], grad_fn=<UnbindBackward0>)
0.19648398458957672 * tensor([0.8600, 0.5600, 0.5300], grad_fn=<UnbindBackward0>)
tensor(1.0000, grad_fn=<AddBackward0>)
tensor([0.6997, 0.5697, 0.6372], grad_fn=<AddBackward0>)


### Get All attention weights

In [99]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(rounded_embeddings):
    for j, x_j in enumerate(rounded_embeddings):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.4782, 0.6804, 0.7307, 0.4214, 0.9571, 0.6915],
        [0.6804, 1.4820, 1.2832, 0.5296, 1.5662, 1.3932],
        [0.7307, 1.2832, 1.4185, 0.4303, 1.5497, 1.2896],
        [0.4214, 0.5296, 0.4303, 0.5554, 0.8249, 0.5159],
        [0.9571, 1.5662, 1.5497, 0.8249, 1.9974, 1.5448],
        [0.6915, 1.3932, 1.2896, 0.5159, 1.5448, 1.3341]],
       grad_fn=<CopySlices>)


We can achive the same but more efficiently via matrix multiplication

In [100]:
# Using matrix transpose (remember is row * column)
print(rounded_embeddings.shape)
print(rounded_embeddings.T.shape)
print('-------------------')
print(rounded_embeddings)
print('-------------------')
print(rounded_embeddings.T)

torch.Size([6, 3])
torch.Size([3, 6])
-------------------
tensor([[0.2300, 0.3800, 0.5300],
        [1.0000, 0.4600, 0.5200],
        [0.6300, 0.9000, 0.4600],
        [0.1500, 0.0000, 0.7300],
        [0.7300, 0.7100, 0.9800],
        [0.8600, 0.5600, 0.5300]], grad_fn=<DivBackward0>)
-------------------
tensor([[0.2300, 1.0000, 0.6300, 0.1500, 0.7300, 0.8600],
        [0.3800, 0.4600, 0.9000, 0.0000, 0.7100, 0.5600],
        [0.5300, 0.5200, 0.4600, 0.7300, 0.9800, 0.5300]],
       grad_fn=<PermuteBackward0>)


In [101]:
attn_scores = rounded_embeddings @ rounded_embeddings.T
print(attn_scores)

tensor([[0.4782, 0.6804, 0.7307, 0.4214, 0.9571, 0.6915],
        [0.6804, 1.4820, 1.2832, 0.5296, 1.5662, 1.3932],
        [0.7307, 1.2832, 1.4185, 0.4303, 1.5497, 1.2896],
        [0.4214, 0.5296, 0.4303, 0.5554, 0.8249, 0.5159],
        [0.9571, 1.5662, 1.5497, 0.8249, 1.9974, 1.5448],
        [0.6915, 1.3932, 1.2896, 0.5159, 1.5448, 1.3341]],
       grad_fn=<MmBackward0>)


Apply softmax

In [102]:
# dim=-1 is to apply softmax to the last dimenison, in this case rows
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.1368, 0.1675, 0.1761, 0.1293, 0.2209, 0.1694],
        [0.0963, 0.2147, 0.1760, 0.0828, 0.2336, 0.1965],
        [0.1054, 0.1832, 0.2097, 0.0781, 0.2392, 0.1844],
        [0.1457, 0.1624, 0.1470, 0.1666, 0.2181, 0.1602],
        [0.0985, 0.1811, 0.1781, 0.0863, 0.2787, 0.1773],
        [0.1008, 0.2033, 0.1833, 0.0845, 0.2365, 0.1916]],
       grad_fn=<SoftmaxBackward0>)


In [103]:
row_0_sum = sum([0.1403, 0.1365, 0.1915, 0.1552, 0.1659, 0.2106])
print(row_0_sum)
print("All row sums:", attn_weights.sum(dim=-1))

1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


### Compute All context vectors

In [104]:
all_context_vecs = attn_weights @ rounded_embeddings
print(all_context_vecs)
print("Previous 2nd context vector:", context_vec_2)

tensor([[0.6362, 0.5392, 0.6413],
        [0.6997, 0.5697, 0.6372],
        [0.6845, 0.5862, 0.6367],
        [0.6105, 0.5069, 0.6496],
        [0.6848, 0.5782, 0.6584],
        [0.6920, 0.5720, 0.6385]], grad_fn=<MmBackward0>)
Previous 2nd context vector: tensor([0.6997, 0.5697, 0.6372], grad_fn=<AddBackward0>)


## Self Attention

**Weight parameters** are learned coefficients that define the network connections, while **attention weights** are dynamic, context-specific values.

In [105]:
x_2 = rounded_embeddings[1] # second input element
d_in = rounded_embeddings.shape[1] #depends on the embeddings dimension the input embedding size, dim=3
d_out = 2 # the output embedding size, dim=2

torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

print(W_query)
print(W_key)
print(W_value)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [106]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

# calculate keys and values vector for all inputs
keys = rounded_embeddings @ W_key
values = rounded_embeddings @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

tensor([0.4503, 1.2839], grad_fn=<SqueezeBackward4>)
keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [107]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.1928, grad_fn=<DotBackward0>)


In [108]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([0.9731, 1.1928, 1.5068, 0.7766, 1.8657, 1.2776],
       grad_fn=<SqueezeBackward4>)


The difference to earlier is that we now scale the attention scores by dividing them by the square root of the embedding dimension,  𝑑𝑘‾‾‾√  (i.e., d_k**0.5):

Imagine you have two vectors, and their dot product results in a large value. When this large value is passed through the softmax function, it might dominate the probabilities, making the attention mechanism less sensitive to other relevant parts of the input. Scaling helps to mitigate this issue by preventing any single dot product from becoming overly influential.

In [109]:
d_k = keys.shape[1] # dimension of keys
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1313, 0.1534, 0.1915, 0.1143, 0.2468, 0.1628],
       grad_fn=<SoftmaxBackward0>)


In [110]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3024, 0.8877], grad_fn=<SqueezeBackward4>)


### Compact SelfAttention Class

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- We can streamline the implementation above using PyTorch's Linear layers instead of torch random, which are equivalent to a matrix multiplication if we disable the bias units.
- In the original Transformer paper, the authors noted that they did not observe significant performance gains from including bias terms in these specific layers. This observation has led to a common practice of omitting bias in similar attention-based architectures.
- Another big advantage of using `nn.Linear` over our manual `nn.Parameter(torch.rand(...)` approach is that `nn.Linear` has a preferred weight initialization scheme, which leads to more stable model training

In [111]:
import torch.nn as nn

In [112]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [113]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(rounded_embeddings))

tensor([[-0.0415,  0.1422],
        [-0.0414,  0.1425],
        [-0.0416,  0.1422],
        [-0.0412,  0.1425],
        [-0.0410,  0.1433],
        [-0.0414,  0.1425]], grad_fn=<MmBackward0>)


### Hiding futer words with causal attention (one step back)

In [114]:
# Reuse data from previous section
queries = sa_v2.W_query(rounded_embeddings)
keys = sa_v2.W_key(rounded_embeddings)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1596, 0.1724, 0.1542, 0.1718, 0.1743, 0.1676],
        [0.1558, 0.1746, 0.1480, 0.1746, 0.1795, 0.1676],
        [0.1551, 0.1740, 0.1468, 0.1752, 0.1818, 0.1670],
        [0.1609, 0.1728, 0.1564, 0.1707, 0.1708, 0.1684],
        [0.1522, 0.1779, 0.1420, 0.1771, 0.1826, 0.1682],
        [0.1557, 0.1745, 0.1478, 0.1747, 0.1798, 0.1675]],
       grad_fn=<SoftmaxBackward0>)


Applying negative infinity effectively zeros out the probabilities for these future tokens in the subsequent softmax calculation.

In [115]:
context_length = attn_scores.shape[-1]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.0899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1507, 0.3114,   -inf,   -inf,   -inf,   -inf],
        [0.1788, 0.3420, 0.1016,   -inf,   -inf,   -inf],
        [0.0464, 0.1474, 0.0061, 0.1298,   -inf,   -inf],
        [0.1874, 0.4080, 0.0893, 0.4012, 0.4449,   -inf],
        [0.1546, 0.3154, 0.0810, 0.3171, 0.3577, 0.2577]],
       grad_fn=<MaskedFillBackward0>)


In [116]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4716, 0.5284, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3258, 0.3657, 0.3085, 0.0000, 0.0000, 0.0000],
        [0.2435, 0.2615, 0.2367, 0.2583, 0.0000, 0.0000],
        [0.1830, 0.2139, 0.1707, 0.2129, 0.2195, 0.0000],
        [0.1557, 0.1745, 0.1478, 0.1747, 0.1798, 0.1675]],
       grad_fn=<SoftmaxBackward0>)


### Masking additional attention weights with dropout

In [117]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))
print(dropout(attn_weights))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6170, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5231, 0.0000, 0.5166, 0.0000, 0.0000],
        [0.0000, 0.4278, 0.3415, 0.4257, 0.4391, 0.0000],
        [0.3114, 0.3490, 0.0000, 0.0000, 0.3596, 0.3350]],
       grad_fn=<MulBackward0>)


### Causal Attention Class

In [118]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [119]:
torch.manual_seed(123)

batch = torch.stack((rounded_embeddings, rounded_embeddings), dim=0)
print(batch.shape) # 2 batches with 6 tokens each, and each token has embedding dimension 3
print(batch)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print("context_shape", context_vecs.shape)
print("context_vecs:", context_vecs)

torch.Size([2, 6, 3])
tensor([[[0.2300, 0.3800, 0.5300],
         [1.0000, 0.4600, 0.5200],
         [0.6300, 0.9000, 0.4600],
         [0.1500, 0.0000, 0.7300],
         [0.7300, 0.7100, 0.9800],
         [0.8600, 0.5600, 0.5300]],

        [[0.2300, 0.3800, 0.5300],
         [1.0000, 0.4600, 0.5200],
         [0.6300, 0.9000, 0.4600],
         [0.1500, 0.0000, 0.7300],
         [0.7300, 0.7100, 0.9800],
         [0.8600, 0.5600, 0.5300]]], grad_fn=<StackBackward0>)
context_shape torch.Size([2, 6, 2])
context_vecs: tensor([[[-0.3582,  0.0069],
         [-0.5720, -0.0601],
         [-0.6188, -0.1391],
         [-0.5268, -0.0428],
         [-0.5956, -0.0437],
         [-0.6151, -0.0593]],

        [[-0.3582,  0.0069],
         [-0.5720, -0.0601],
         [-0.6188, -0.1391],
         [-0.5268, -0.0428],
         [-0.5956, -0.0437],
         [-0.6151, -0.0593]]], grad_fn=<UnsafeViewBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

## Self Attention multi-head

In [121]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.3582,  0.0069,  0.3515,  0.1966],
         [-0.5720, -0.0601,  0.5608,  0.3132],
         [-0.6188, -0.1391,  0.6004,  0.3960],
         [-0.5268, -0.0428,  0.5252,  0.3050],
         [-0.5956, -0.0437,  0.5882,  0.3421],
         [-0.6151, -0.0593,  0.6044,  0.3508]],

        [[-0.3582,  0.0069,  0.3515,  0.1966],
         [-0.5720, -0.0601,  0.5608,  0.3132],
         [-0.6188, -0.1391,  0.6004,  0.3960],
         [-0.5268, -0.0428,  0.5252,  0.3050],
         [-0.5956, -0.0437,  0.5882,  0.3421],
         [-0.6151, -0.0593,  0.6044,  0.3508]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- In the implementation above, the embedding dimension is 4, because we `d_out=2` as the embedding dimension for the key, query, and value vectors as well as the context vector. And since we have 2 attention heads, we have the output embedding dimension 2*2=4

In [122]:
# Multiple heads to extract differente type of information, every head is using different initialized weights
# cada head da como resultado context vectors de cierte dimension que al final son concatenados

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
                             "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # b is for batches
        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)
        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.2547, 0.5044],
         [0.2759, 0.3883],
         [0.2662, 0.3538],
         [0.2737, 0.4130],
         [0.2861, 0.3772],
         [0.2848, 0.3659]],

        [[0.2547, 0.5044],
         [0.2759, 0.3883],
         [0.2662, 0.3538],
         [0.2737, 0.4130],
         [0.2861, 0.3772],
         [0.2848, 0.3659]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
